In [1]:
from sentence_transformers import SentenceTransformer

# Load your model
model = SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1")

/opt/homebrew/anaconda3/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
model.encode("Hello, world!")

array([ 0.3059689 ,  0.79072964,  0.00980721, ...,  0.0644002 ,
       -0.45898244, -0.01831866], dtype=float32)

In [3]:
import batch
model.forward = batch.dynamically(model.forward, device="mps")

In [4]:
from concurrent.futures import ThreadPoolExecutor

def encode_text(text):
    return model.encode(text)

with ThreadPoolExecutor(max_workers=128) as executor:
    futures = [executor.submit(encode_text, "Hello, world!") for _ in range(100)]
    results = [future.result() for future in futures]

Exception in thread Thread-7 (_process_batches):
Traceback (most recent call last):
  File "/opt/homebrew/anaconda3/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/opt/homebrew/anaconda3/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/opt/homebrew/anaconda3/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/rui/Documents/batch/batch/batch_processor.py", line 103, in _process_batches
    else self._inference_func(raw_batch)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/anaconda3/lib/python3.11/site-packages/torch/nn/modules/container.py", line 217, in forward
    input = module(input)
            ^^^^^^^^^^^^^
  File "/opt/homebrew/anaconda3/lib/python3.11/site-packages/torch/nn/modules/module.py", line 1532, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^

KeyboardInterrupt: 

In [23]:
for i in range(100):
    model.encode("Hello, world!")

In [6]:
import inspect

inspect.signature(model.forward).parameters["input"]

KeyError: 'input'

In [31]:
from ofen.models import TextEncoder

text_encoder = TextEncoder("mixedbread-ai/mxbai-embed-large-v1")
text_encoder.encode("Hello, world!")

/opt/homebrew/anaconda3/lib/python3.11/site-packages/ofen/models/torch/text_encoder.py:144: UserWarning: Pooling strategy is set to NONE. This may result in unexpected behavior as the output will be the full sequence of hidden states rather than a single vector per input. Consider using a different pooling strategy such as mean, max, or cls if you need a fixed-size representation.
  warnings.warn(


EncodingResult(embeddings=array([[[ 0.01729078,  0.04468535,  0.00055422, ...,  0.00363935,
         -0.02593779, -0.00103519],
        [-0.00385487,  0.02841955,  0.00728079, ..., -0.00641318,
         -0.00887359,  0.01458311],
        [ 0.02255957,  0.03204089,  0.01209311, ...,  0.00168585,
         -0.03093472,  0.01115617],
        [-0.01648994,  0.0239604 , -0.01059136, ..., -0.01044505,
         -0.04768615,  0.02494777],
        [ 0.00466857,  0.04424843,  0.00212991, ...,  0.01685552,
         -0.01741115,  0.01215234],
        [ 0.00623544,  0.03078753,  0.00834456, ...,  0.00311965,
         -0.0418628 , -0.02152617]]], dtype=float32), total_tokens=6)

In [32]:
text_encoder.forward = batch.dynamically(text_encoder.forward)

In [50]:
from concurrent.futures import ThreadPoolExecutor

def encode_text(text):
    return text_encoder.encode(text)

with ThreadPoolExecutor(max_workers=128) as executor:
    futures = [executor.submit(encode_text, "Hello, world!") for _ in range(100)]
    results = [future.result() for future in futures]